# CM50270 Reinforcement Learning
## Coursework Part 3: Tic-Tac-Toe

In this piece of coursework, you will implement the game of Tic-Tac-Toe and a Q-Learning agent capable of playing it.

**Total number of marks:** 40 marks.

**What to submit:** Your completed Jupyter notebook (.ipynb file) which should include all of your source code. Please do not change the file name or compress/zip your submission. Please do not include any identifying information on the files you submit. This coursework will be **marked anonymously**.

**Where to submit:** CM50270 Moodle Page.

You are required to **work individually**. You are welcome to discuss ideas with others but you must design your own implementation and write your own code.

**Do not plagiarise**. Plagiarism is a serious academic offence. For details on what plagiarism is and how to avoid it, please visit the following webpage: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html

If you are asked to use specific variable names, data-types, function signatures and notebook cells, **please ensure that you follow these instructions**. Not doing so will cause the automarker to reject your work, and will assign you a score of zero for that question. **If the automarker rejects your work because you have not followed the instructions, you may not get any credit for your work**.

Please remember to **save your work regularly**.

Please be sure to **restart the kernel and run your code from start-to-finish** (Kernel → Restart & Run All) before submitting your notebook. Otherwise, you may not be aware that you are using variables in memory that you have deleted.

Your total runtime must be less than **3 minutes** on the University's computers in 10W 0.02 and that **written answer length limits** are adhered to. Otherwise, you may not get credit for your work.

## Exercise 1: Tic-Tac-Toe Environment
For this exercise, you should implement the game of [Tic-Tac-Toe](https://en.wikipedia.org/wiki/Tic-tac-toe) (also known as Noughts and Crosses). After you have implemented the environment, you will validate it by letting two random agents play against each other.

### The Tic-Tac-Toe Environment

Tic-Tac-Toe is a simple game for two players, O and X, who take turns marking cells in a 3x3 grid. The player who succeeds in drawing three instances of their symbol in a horizontal, vertical or diagonal line wins the game. The following example, from [Wikipedia](https://commons.wikimedia.org/wiki/File:Tic-tac-toe-game-1.svg), shows a game that is won by the X player.

<img src="images/tic-tac-toe_example.svg" style="width: 600px;"/> 

### Instructions
Implement the game of Tic-Tac-Toe. The first-moving player should be randomly selected at the beginning of each episode. Player X should always be a random agent (i.e. it places an X symbol in a random empty grid-space each time-step). You will implement various agents for player O. Rewards of +1, -1 and 0 should be collected for winning, losing and drawing respectively. All other rewards collected during the game should be 0.

Test your Tic-Tac-Toe implementation by letting two random agents play against each other for 100 episodes. Plot the **cumulative rewards** collected by both the O and X agents as a function of the number of episodes.

Hint: The X player can be considered to be part of the environment, as it is outside the agent's control.

**Exercises 1, 2, and 4 will be marked jointly; you will see this joint score in Exercise 4. You will be given marks for the strength of your implementation and your understanding of the techniques you have implemented. Please note that the standard implementation of Q-learning will receive at most 8 marks from exercise 3 and 0 marks from exercises 1, 2, and 4. This is the most challenging part of your coursework. Precise instructions are given below.**

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output
from collections import defaultdict
import time

In [ ]:
class TicTacToe(object):
    
    def __init__(self):
        self.gameboard = np.zeros((3, 3))
        self.players = [0, 1]
        self.first_player = np.random.choice(self.players)
   
    def reset(self):
        self.gameboard = np.zeros((3, 3))
        self.first_player = np.random.choice(self.players)
        
    def is_winning_state(self, state):
        hor_win = 3 in np.sum(state, axis=1)
        vert_win = 3 in np.sum(state, axis=0)
        diag_win = 3 == np.trace(state)
        antidiag_win = 3 == np.trace(state[:,-1::-1])
        is_win = hor_win or vert_win or diag_win or antidiag_win
        return is_win
    
    def is_losing_state(self, state):
        hor_loss = -3 in np.sum(state, axis=1)
        vert_loss = -3 in np.sum(state, axis=0)
        diag_loss = -3 == np.trace(state)
        antidiag_loss = -3 == np.trace(state[:,-1::-1])
        is_loss = hor_loss or vert_loss or diag_loss or antidiag_loss
        return is_loss
        
    def is_tie_state(self, state):
        if not 0 in state and not (self.is_winning_state(state) or self.is_losing_state(state)):
            return True
        else:
            return False
        
    def is_final_state(self, state):
        if not (self.is_losing_state(state) or self.is_winning_state(state) or self.is_tie_state(state)):
            return False
        else:
            return True
    
    def get_possible_actions(self, state):
        possible_actions = [(i, j) for i in range(3) for j in range(3) if state[i, j]==0]
        return possible_actions
             
    def step(self, state, action, player):
        reward = np.array([0, 0])
        terminal = False
        new_state = np.copy(state)
        new_state[action] = (-1)**player
        if self.is_winning_state(new_state):
            reward = np.array([1, -1])
            terminal = True
        elif self.is_losing_state(new_state):
            reward = np.array([-1, 1])
            terminal = True
        elif self.is_tie_state(new_state):
            terminal = True
        return new_state, reward, terminal

In [ ]:
class Random_Agent(object):
    def choose_action(self, state, available_actions):
        action_index = np.random.randint(0, len(available_actions))
        action = available_actions[action_index]
        return action
    def learn(self, old_state, reward, new_state, action):
        pass

In [ ]:
def play_tictactoe(game, agent1, agent2=Random_Agent() ,n_episodes=100):
    returns_1 = np.zeros(n_episodes)
    returns_2 = np.zeros(n_episodes)
    agents = {0: agent1, 1: agent2}
    cu_reward_1 = 0
    cu_reward_2 = 0
    for episode in range(n_episodes):
        game_ended = False
        first_player = game.first_player
        second_player = 1 - first_player
        step = 0
        while not game_ended:
            
            #perform step by first player
            old_state = np.copy(game.gameboard)
            first_available_actions = game.get_possible_actions(old_state)
            first_action=agents[first_player].choose_action(old_state, first_available_actions)
            first_state, first_reward, first_terminal = game.step(old_state, first_action, first_player)
            
            #if game has already progressed one or more moves, update second player by using previous move
            if step>=1:
                agents[second_player].learn(first_state_prev, first_reward[second_player], first_state, second_action)
                
            #check if game has ended after first player's move
            if first_terminal:
                game_ended=True
                cu_reward_1 += first_reward[0]
                cu_reward_2 += first_reward[1] 
                agents[first_player].learn(old_state, first_reward[first_player], first_state, first_action)
                game.reset()
                break
                
            #if not ended, perform second player's move
            second_available_actions = game.get_possible_actions(first_state)
            second_action = agents[second_player].choose_action(first_state, second_available_actions)
            second_state, second_reward, second_terminal = game.step(first_state, second_action, second_player)
            
            #train first agent
            agents[first_player].learn(old_state, second_reward[first_player], second_state, first_action)
            
            #check if game has ended after second player performed his move
            if second_terminal:
                game_ended=True
                cu_reward_1 += second_reward[0]
                cu_reward_2 += second_reward[1] 
                agents[second_player].learn(first_state, second_reward[second_player], second_state, second_action)
                game.reset()
                break
                
            #update gameboard 
            step+=1
            game.gameboard = second_state
            first_state_prev = np.copy(first_state)
            
        returns_1[episode] = cu_reward_1
        returns_2[episode] = cu_reward_2
                 
    return returns_1, returns_2

In [ ]:
# Please write your code for Exercise 1 in this cell or in as many cells as you want ABOVE this cell.
# You should implement your Tic-Tac-Toe Environment, your random O agent, and produce your cumulative reward plot.
# Do NOT delete this cell.

# YOUR CODE HERE
n_episodes = 100
game = TicTacToe()
player1 = Random_Agent()
episodes = np.arange(0, n_episodes, 1, dtype=int)
r1, r2=play_tictactoe(game, player1, n_episodes=n_episodes)

fig=plt.figure(figsize=(10, 6))
ax = fig.add_subplot("111")
ax.set_title("Cumulative rewards of two random agents playing against another")
ax.plot(episodes, r1, label="Random Agent 1")
ax.plot(episodes, r2, label="Random Agent 2")
ax.set_xlabel("Episode")
ax.set_ylabel("Cum. reward")
plt.legend()
plt.show()

## Exercise 2: Q-Learning Agent
In this exercise, you will implement an agent which learns an optimal policy for playing Tic-Tac-Toe against a random opponent using Q-Learning. For your reference, the pseudocode for the Q-Learning algorithm is reproduced below (Reinforcement Learning, Sutton & Barto, 2018, Section 6.5 p.131).

<img src="images/q_learning_algo.png" style="width: 650px;"/>

In order to score high marks in this coursework, you will need to extend your solution beyond a simple Q-Learning agent to achieve more efficient learning (i.e., using fewer interactions with the environment). Ideas for improving your agent will have been discussed in lectures, and can be found in the course textbook (Reinforcement Learning, Sutton & Barto, 2018). However you go about improving your agent, it must still use tabular Q-Learning at its core. 

Please produce the following:
- A Q-Learning agent for the O player, with whatever modifications you believe are reasonable in order to acheieve good performance in the game of Tic-Tac-Toe.
- A learning curve for your agent. This learning curve should plot the performance of a single agent, and should be smoothed by taking the mean of the cumulative rewards collected by the agent every 20 episodes. You should not smooth your learning curve in any other way.


In [ ]:
class QLearning_Agent(object):
    def __init__(self, game, player, alpha=0.2, gamma=0.9, eps=0.1):
        self.alpha = alpha
        self.eps = eps
        self.gamma = gamma
        self.player = player
        self.game = game
        self.q_values = defaultdict(lambda: 0, {})
    
    def rotate_90(self, state, action):
        s_rot90 = np.array([[state[2-j,i] for j in range(3)] for i in range(3)])
        a_rot90 = (2-action[1],action[0])
        sa_rot90 = (s_rot90, a_rot90)
        return sa_rot90
        
    def calc_symmetries(self, state, action):
        #reflect along the vertical
        s_v = state[:, -1::-1]
        a_v = (action[0], 2 - action[1])
        sa_v = (s_v, a_v)
        #reflect along the horizontal
        s_h = state[-1::-1,:]
        a_h = (2 - action[0], action[1])
        sa_h = (s_h, a_h)
        #reflect along the left-right diagonal
        s_lr = state[-1::-1,-1::-1].transpose()
        a_lr = (2-action[1], 2-action[0])
        sa_lr = (s_lr, a_lr)
        #reflect along the right-left diagonal
        s_rl = state.transpose()
        a_rl = (action[1], action[0])
        sa_rl = (s_rl, a_rl)
        #rotate by 90 degrees
        sa_rot90 = self.rotate_90(state, action)
        #rotate by 180 degrees
        sa_rot180 = self.rotate_90(sa_rot90[0], sa_rot90[1])
        #rotate by 270 degrees
        sa_rot270 = self.rotate_90(sa_rot180[0], sa_rot180[1])
        
        return [sa_v, sa_h, sa_]
    
    def choose_action(self, state, available_actions):
        n_actions = len(available_actions)
        q_action = [self.q_values[(str(state), action)] for action in available_actions]
        max_q_value = max(q_action)
        optimal_action_indices = [i for i in range(n_actions) if q_action[i]==max_q_value]
        action_index = np.random.choice(optimal_action_indices)
        if np.random.uniform()<self.eps and len(available_actions)>1:
            action_index = np.random.choice(np.delete([i for i in range(n_actions)], action_index))
        action = available_actions[action_index]
        return action
    
    def learn(self, old_state, reward, new_state, action):
        #calculate all symmetric states
        symmetric_state_action_pairs = self.calc_symmetries(old_state, action)
        state_action_pairs = symmetric_state_action_pairs + [(old_state, action)]
        #state_action_pairs = [(old_state, action)]
        #calculate new q-value
        available_actions = game.get_possible_actions(new_state)
        q_old = self.q_values[(str(old_state), action)]
        q_action = [self.q_values[(str(new_state), a)] for a in available_actions]
        #if new state is terminal
        if game.is_final_state(new_state):
            q_max = 0
        else:
            q_max = max(q_action)
        q_new = q_old + self.alpha*(reward + self.gamma*q_max - q_old)
        #update q-values of all symmetric states
        for state_action in state_action_pairs:
            state_, action_ = state_action
            if game.is_final_state(old_state):
                raise ValueError("Updating final state!")
            self.q_values[(str(state_), action_)] = q_new
        

In [ ]:
n_episodes = 10000
game = TicTacToe()
player1 = QLearning_Agent(game, 0)
player2 = Random_Agent()
episodes = np.arange(0, n_episodes, 1, dtype=int)
r1, r2=play_tictactoe(game, player1, player2, n_episodes=n_episodes)

In [ ]:
def running_mean(data, window_size):
    mean = np.zeros(len(data))
    for i in range(len(data)//window_size):
        current_mean = np.sum(data[i*window_size:(i+1)*window_size])/window_size
        mean[i*window_size:(i+1)*window_size] = current_mean
    return mean

In [ ]:
# Please write your code for Exercise 2 in this cell or in as many cells as you want ABOVE this cell.
# You should implement your Q-Learning agent and produce your average learning curve plot.
# Do NOT delete this cell.

# YOUR CODE HERE

episodes = np.arange(0, n_episodes, 1)
y1=running_mean(r1, 20)
y2=running_mean(r2, 20)

fig=plt.figure(figsize=(10, 6))
ax = fig.add_subplot("111")
ax.set_title("Cumulative rewards of a Q-Learning agent playing against a random agent")
ax.plot(episodes, y1, label="Q-Learning Agent")
ax.plot(episodes, y2, label="Random Agent")
ax.set_xlabel("Episode")
ax.set_ylabel("Cum. reward")
plt.legend()
plt.show()

## Exercise 3: Optimal Policy (8 Marks)
In this exercise, we will test your agent's policy to see whether it has learned an optimal policy.

Please define a function `test_policy(state)` below, which takes a state and returns the action that your agent would take in that state. **This function should not re-train an agent every time it is called**, it should use the policy that was learned by the agent that you trained in the previous exercise.

We will represent the state as a list of nine characters, either `'X'`, `'O'` or `''` for cells occupied by X, O, an neither player respectively. The list index corresponding to each of the nine grid-cells is as follows:

<img src="images/tic_tac_toe_grid_indexes.png" style="width: 200px;"/>

The integer your `test_policy(state)` function returns should be the index corresponding to the cell your agent would place its O symbol in.

We will be testing a number of states. Among them is the following state:

<img src="images/tic_tac_toe_question_state.png" style="width: 200px;"/>

This state will be passed into your `test_policy(state)` function as the list `['O', 'X', '', '', 'X', '', '', 'O', '']`. If, for example, your agent's policy is to place its symbol in the top-right cell, your function should return the integer value `2`.


In [ ]:
# Please write your code for Exercise 3 in this cell or in as many cells as you want ABOVE this cell.
# You should implement your test_policy(state) function here.
# Do NOT delete this cell.
# YOUR CODE HERE
def test_policy(state) :

    state_array = np.zeros((3,3))
    for index, symbol in enumerate(state):
        i,j = index//3, index%3
        if symbol=='O':
            state_array[i,j]=1
        elif symbol=='X':
            state_array[i,j]=-1
        elif symbol=='':
            state_array[i,j]=0
        else:
            raise ValueError("Invalid symbol in array:",symbol)
    available_actions = game.get_possible_actions(state_array)
    print(state_array)
    q_actions = [player1.q_values[(str(state_array), action)] for action in available_actions]
    print(max(q_actions))
    best_action_index = np.argmax(q_actions)
    best_action = available_actions[best_action_index]
    index = 3*best_action[0] + best_action[1]
    return index    
test_policy(['O','X','',
            '','','',
            '','',''])

In [ ]:
a = np.random.randint(0, 10, size=(3,3))
print(a)
print(a[-1::-1, -1::-1].transpose())

In [ ]:
# DO NOT DELETE THIS CELL!
# Your code for Exercise 3 is tested here.


In [ ]:
# DO NOT DELETE THIS CELL!
# Your code for Exercise 3 is tested here.


In [ ]:
# DO NOT DELETE THIS CELL!
# Your code for Exercise 3 is tested here.


In [ ]:
# DO NOT DELETE THIS CELL!
# Your code for Exercise 3 is tested here.


## Exercise 4: Discussion (32 Marks)

In eight sentences or fewer, please discuss the following:
- Your chosen state representation.
- Any additions that you have made to your implementation beyond implementing basic Q-Learning.
- The effects you expected your additions to have, and the extent to which your expectations were met.
- Further modifications you believe may enhance the performance of your agent, or changes you would make if you had more time.

For the task of teaching a Q-Learning agent to learn an optimal policy playing Tic-Tac-Toe against a random agent, the following state representation was used: The state representing the game status was a 3x3 array, with 0 representing an empty field, 1 standing for a field with a "O" on it and -1 representing a field with a "X" on it.

In addition to the normal Q-Learning implementation following additions were made to the Q-Learning agent:
Whenever a Q-value for a state-action pair was updated, the same update was performed to all symmetric state-value pairs. Tic-Tac-Toe states are symmetric with respect to horizontal and vertical reflections and rotations by 90, 180 and 270 degrees.
So each time the Q-value of a specific state-action pair is updated, all the symmetric state-action pairs are calculated and their respective values in the Q-table are updated as well. 

My expectation was that this would lead to the Q-agent learning more quickly, because it allowed it to improve its policy for states that it did not explicitly experience during the training, or at least not as often. This expectation was met to some extent, as the reward accumulated by the agent increased earlier in the training, as opposed to it learning naively. However, as each time a state-action pair was updated its symmetries were explicitly calculated, this led to a dent in performance. It might have been better to save these state-action pairs once they had been calculated in a dictionary and simply look them up as it became necessary (leading to an increase in memory-usage).

Further modifications, which might have improved the agent's learning performance: Identifying state-action pairs resulting in the same so-called afterstate and simultaneously updating their Q-values would have also lead to an increase in the performance of the agent, but this was not implemented in this version.